In [ ]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct
import os

def read_image(image_path):
    # Read image and convert it to YCbCr (YUV) color space.
    image = Image.open(image_path)
    image_ycbcr = image.convert('YCbCr')
    img_array = np.array(image_ycbcr, dtype=float)
    return img_array

def chroma_subsampling(Cb, Cr):
    # 4:2:0 Chroma Subsampling
    Cb_subsampled = Cb[::2, ::2]
    Cr_subsampled = Cr[::2, ::2]
    return Cb_subsampled, Cr_subsampled

def process_channel_as_block(channel, blocksize=8, blockFunction = None):
    # Dimensions of the image / chanel
    h, w = channel.shape
    
    # Add Padding so that the image can be divided into blocks of 8 x 8 without any "leftovers"
    h = (blocksize - (h % blocksize) ) % blocksize
    w = (blocksize - (w % blocksize) ) % blocksize
    channel = np.pad(channel, ((0, h), (0, w)), mode='constant', constant_values=0) # set the values of the padding to 0

    transformedChanel = np.zeros(channel, dtype=float)
    for i in range(0, channel.shape[0], blocksize):
        for j in range(0, channel.shape[1], blocksize):
            block = channel[i: i+blocksize, j: j+blocksize]
            
            if blockFunction is None: 
                raise RuntimeError("No DCT transformation function has been provided!")
            
            transformedBlock = blockFunction(block)
            transformedChanel[i: i+blocksize, j: j+blocksize] = transformedBlock

    # Remove the overflow-padding that was added at the beginning
    return transformedChanel[:h, :w] 


def dct_2d_block(block):
    # Apply the two-dimensional dct transformation to a block of an image
    return dct(dct(block.T, norm='ortho').T, norm='ortho')


def eliminate_dct_coefficients(dct_block, removal_percentage):
    """
    Eliminates a certain percentage of the DCT coefficients of a block.
    
    - removal_percentage: The percentage of coefficients that should be removed.
    """
    coefficients = dct_block.flatten()
    amount_of_coefficients = len(coefficients)
    
    # Determinate the amount of coefficients that remain
    remaining_coefficients = int(amount_of_coefficients * (1 - removal_percentage))
    
    # Coefficient "Removal", set to-be-removed coefficients to 0 
    coefficients[remaining_coefficients: ] = 0
    
    return coefficients.reshape(dct_block.shape)
    

def main(): 
    image_path = "OriginalImage.png"
    img_channels = read_image(image_path)
    
    Y = img_channels[:, :, 0]
    Cb = img_channels[:, :, 1]
    Cr = img_channels[:, :, 2]
    
    # 4:2:0 Chroma Subsampling
    Cb_sampled, Cr_sampled = chroma_subsampling(Cb, Cr)


    # DCT - with blocks with a size of 8 x 8
    Y_dct = process_channel_as_block(Y, blockFunction=dct_2d_block)
    Cb_dct = process_channel_as_block(Cb_sampled, blockFunction=dct_2d_block)
    Cr_dct = process_channel_as_block(Cr_sampled, blockFunction=dct_2d_block)
    
    
    
    
    
    
